In [2]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
# Load the dataset

df = pd.read_csv(r'C:\Users\anant\OneDrive\Desktop\MBAN\MBAN 6110\Datasets\Final Presentation\train.csv')

C:\Users\anant\AppData\Local\Temp\ipykernel_21992\2144700389.py:3: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\anant\OneDrive\Desktop\MBAN\MBAN 6110\Datasets\Final Presentation\train.csv')


In [3]:
# Display the first few rows of the dataset

df.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529544,High_spent_Small_value_payments,312.4940887,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.2802216,Low_spent_Large_value_payments,284.6291625,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.69952126,Low_spent_Medium_value_payments,331.2098629,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580744,Low_spent_Small_value_payments,223.4513097,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.42015309,High_spent_Medium_value_payments,341.489231,Good


In [4]:
# Check for missing values

df.isnull().sum()

ID                              0
Customer_ID                     0
Month                           0
Name                         9985
Age                             0
SSN                             0
Occupation                      0
Annual_Income                   0
Monthly_Inhand_Salary       15002
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                11408
Delay_from_due_date             0
Num_of_Delayed_Payment       7002
Changed_Credit_Limit            0
Num_Credit_Inquiries         1965
Credit_Mix                      0
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age           9030
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly      4479
Payment_Behaviour               0
Monthly_Balance              1200
Credit_Score                    0
dtype: int64

In [5]:
# Check unique values in each categorical column

df.select_dtypes(include=['object']).nunique()

ID                         100000
Customer_ID                 12500
Month                           8
Name                        10139
Age                          1788
SSN                         12501
Occupation                     16
Annual_Income               18940
Num_of_Loan                   434
Type_of_Loan                 6260
Num_of_Delayed_Payment        749
Changed_Credit_Limit         3635
Credit_Mix                      4
Outstanding_Debt            13178
Credit_History_Age            404
Payment_of_Min_Amount           3
Amount_invested_monthly     91049
Payment_Behaviour               7
Monthly_Balance             98790
Credit_Score                    3
dtype: int64

In [6]:
# Get descriptive statistics for each numerical column

df.describe()

,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Delay_from_due_date,Num_Credit_Inquiries,Credit_Utilization_Ratio,Total_EMI_per_month
count,84998.000000,100000.000000,100000.00000,100000.000000,100000.000000,98035.000000,100000.000000,100000.000000
mean,4194.170850,17.091280,22.47443,72.466040,21.068780,27.754251,32.285173,1403.118217
std,3183.686167,117.404834,129.05741,466.422621,14.860104,193.177339,5.116875,8306.041270
min,303.645417,-1.000000,0.00000,1.000000,-5.000000,0.000000,20.000000,0.000000
25%,1625.568229,3.000000,4.00000,8.000000,10.000000,3.000000,28.052567,30.306660
50%,3093.745000,6.000000,5.00000,13.000000,18.000000,6.000000,32.305784,69.249473
75%,5957.448333,7.000000,7.00000,20.000000,28.000000,9.000000,36.496663,161.224249
max,15204.633330,1798.000000,1499.00000,5797.000000,67.000000,2597.000000,50.000000,82331.000000


In [4]:
# Convert Age column to numeric and fill in missing or erroneous ages

df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
df['Age'] = df['Age'].apply(lambda x: x if 0 < x < 100 else None)
df['Age'] = df.groupby('Customer_ID')['Age'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else x.mean()))

In [5]:
# Fill in missing customer names

df['Name'] = df.groupby('Customer_ID')['Name'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else "Unknown"))

In [6]:
# Fill in missing occupations

df['Occupation'] = df.groupby('Customer_ID')['Occupation'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else "Unknown"))

In [7]:
# Clean and convert Annual_Income and Monthly_Inhand_Salary columns

for column in ['Annual_Income', 'Monthly_Inhand_Salary']:
    df[column] = df[column].replace('[^0-9.]','', regex=True)
    df[column] = pd.to_numeric(df[column], errors='coerce')

In [8]:
# Clean and convert a list of numerical columns

num_cols = ['Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 
            'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 
            'Num_Credit_Inquiries', 'Outstanding_Debt', 'Credit_Utilization_Ratio', 
            'Total_EMI_per_month', 'Amount_invested_monthly', 'Monthly_Balance']


for column in num_cols:
    df[column] = df[column].replace('[^0-9.]','', regex=True)
    df[column] = pd.to_numeric(df[column], errors='coerce')

In [9]:
# Fill in missing or changed Annual_Income based on occupation

mode_annual_income = df.groupby('Customer_ID')['Annual_Income'].apply(lambda x: x.mode()[0] if not x.mode().empty else None).reset_index()
mode_annual_income.columns = ['Customer_ID', 'Mode_Annual_Income']

df = pd.merge(df, mode_annual_income, on='Customer_ID', how='left')

df.loc[(df['Annual_Income'].isna()) & (df['Occupation'] == df['Occupation'].shift()), 'Annual_Income'] = df['Mode_Annual_Income']
df.loc[(df['Annual_Income'].isna()) & (df['Occupation'] != df['Occupation'].shift()), 'Annual_Income'] = df['Annual_Income'].fillna(method='ffill')

df.drop('Mode_Annual_Income', axis=1, inplace=True)

In [10]:
# Fill in missing Monthly_Inhand_Salary based on customer ID

mode_monthly_salary = df.groupby('Customer_ID')['Monthly_Inhand_Salary'].apply(lambda x: x.mode()[0] if not x.mode().empty else None).reset_index()
mode_monthly_salary.columns = ['Customer_ID', 'Mode_Monthly_Salary']

df = pd.merge(df, mode_monthly_salary, on='Customer_ID', how='left')

df.loc[df['Monthly_Inhand_Salary'].isna(), 'Monthly_Inhand_Salary'] = df['Mode_Monthly_Salary']

df.drop('Mode_Monthly_Salary', axis=1, inplace=True)

In [11]:
# Clean Type_of_Loan column

df['Type_of_Loan'] = df.groupby('Customer_ID')['Type_of_Loan'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else "No Loan"))
df['Type_of_Loan'] = df['Type_of_Loan'].apply(lambda x: [i.strip() for i in x.replace("and", ",").split(",") if i.strip()])

for loan in set([j for i in df['Type_of_Loan'].tolist() for j in i]):
    df[loan] = df['Type_of_Loan'].apply(lambda x: 1 if loan in x else 0)

In [12]:
# Rename 'Not Specified' column to 'Other'

df.rename(columns={'Not Specified': 'Other'}, inplace=True)

In [13]:
# Clean Credit_Mix column

df['Credit_Mix'] = df.groupby('Customer_ID')['Credit_Mix'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else "Unknown"))

In [17]:
# Split Credit_History_Age into years and months, calculate the total age in months, and handle missing values

df[['Years', 'Months']] = df['Credit_History_Age'].str.split(' and ', expand=True)
df['Years'] = df['Years'].str.replace(' Years', '').astype(float)
df['Months'] = df['Months'].str.replace(' Months', '').astype(float)
df['Total_Age_in_Months'] = df['Years']*12 + df['Months']

# Convert Total_Age_in_Months to float

df['Total_Age_in_Months'] = df['Total_Age_in_Months'].astype(float)

# Fill NaN values

# Interpolate missing values

df['Total_Age_in_Months'] = df.groupby('Customer_ID')['Total_Age_in_Months'].apply(lambda group: group.interpolate()).reset_index(level=0, drop=True)

# Drop columns

df.drop(['Years', 'Months', 'Credit_History_Age'], axis=1, inplace=True)

In [18]:
# For each numerical column, identify the unique values and their frequencies

unique_values = {column: df.groupby('Customer_ID')[column].nunique() for column in num_cols}

In [19]:
# Identify the columns with erroneous values, i.e., those with more than one unique value per customer ID

erroneous_columns = [column for column in unique_values if any(unique_values[column] > 1)]

In [20]:
# Replace erroneous values in each column with the mode of the customer ID, this is to replace garbage values that could exist

for column in erroneous_columns:
    mode_values = df.groupby('Customer_ID')[column].apply(lambda x: x.mode()[0] if not x.mode().empty else None).reset_index()
    mode_values.columns = ['Customer_ID', 'Mode_Value']
    df = pd.merge(df, mode_values, on='Customer_ID', how='left')
    df.loc[df[column] != df['Mode_Value'], column] = df['Mode_Value']
    df.drop('Mode_Value', axis=1, inplace=True)

In [21]:
# Payment_Behaviour - Replacing  erroneous values with the mode of the customer ID

mode_values = df.groupby('Customer_ID')['Payment_Behaviour'].apply(lambda x: x.mode()[0] if not x.mode().empty else None).reset_index()
mode_values.columns = ['Customer_ID', 'Mode_Value']
df = pd.merge(df, mode_values, on='Customer_ID', how='left')
df.loc[df['Payment_Behaviour'] != df['Mode_Value'], 'Payment_Behaviour'] = df['Mode_Value']
df.drop('Mode_Value', axis=1, inplace=True)

In [24]:
df.to_csv('train_cleaned.csv')

In [28]:
df_cleaned = df.copy()